# Initialisation de Spark

In [1]:
from pyspark import SparkContext, SparkConf
conf = SparkConf().setAppName("Notebook").setMaster("local")
sc = SparkContext(conf=conf)

Using Spark's default log4j profile: org/apache/spark/log4j2-defaults.properties
25/11/10 09:44:14 WARN Utils: Your hostname, Dell, resolves to a loopback address: 127.0.1.1; using 10.255.255.254 instead (on interface lo)
25/11/10 09:44:14 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Using Spark's default log4j profile: org/apache/spark/log4j2-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/11/10 09:44:15 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


# Une fonction pour simplifier l'accès aux données

In [3]:
def extract_field(s: str, field_number: int, sep: str = ';') -> str:
    fields = s.split(sep)
    return fields[field_number] if (field_number < len(fields)) else ""

In [ ]:
print(extract_field("2;CASSIOPEE;2009;33;3", 0))
print(extract_field("2;CASSIOPEE;2009;33;3", 1))
print(extract_field("2;CASSIOPEE;2009;33;3", 2))
print(extract_field("2;CASSIOPEE;2009;33;3", 3))
print(extract_field("2;CASSIOPEE;2009;33;3", 4))
print(extract_field("2;CASSIOPEE;2009;33;3", 5))

2
CASSIOPEE
2009
33
3



# Charger les données
1. Créer le RDD `lignes` à partir du répertoire `prenoms_sample.txt`

In [4]:
lignes = sc.textFile("prenoms_sample.txt")
print(lignes.take(10))

['1;_PRENOMS_RARES;1910;06;38', '1;_PRENOMS_RARES;1919;974;122', '1;_PRENOMS_RARES;1921;84;3', '1;_PRENOMS_RARES;1929;43;7', '1;_PRENOMS_RARES;1934;971;190', '1;_PRENOMS_RARES;1942;85;11', '1;_PRENOMS_RARES;1949;34;10', '1;_PRENOMS_RARES;1967;04;3', '1;_PRENOMS_RARES;1967;08;17', '1;_PRENOMS_RARES;1992;75;1166']


# Transformer les lignes en prénoms
1. En appliquant la méthode `map`, créer le RDD `prenoms` à partir de `lignes`

In [5]:
prenoms = lignes.map(lambda l: (
    extract_field(l, 0)[0],
    extract_field(l, 1),
    int(extract_field(l, 2)),
    int(extract_field(l, 3)),
    int(extract_field(l, 4))
))
for n in prenoms.take(10):
    print(n)

('1', '_PRENOMS_RARES', 1910, 6, 38)
('1', '_PRENOMS_RARES', 1919, 974, 122)
('1', '_PRENOMS_RARES', 1921, 84, 3)
('1', '_PRENOMS_RARES', 1929, 43, 7)
('1', '_PRENOMS_RARES', 1934, 971, 190)
('1', '_PRENOMS_RARES', 1942, 85, 11)
('1', '_PRENOMS_RARES', 1949, 34, 10)
('1', '_PRENOMS_RARES', 1967, 4, 3)
('1', '_PRENOMS_RARES', 1967, 8, 17)
('1', '_PRENOMS_RARES', 1992, 75, 1166)


# Interroger les données
La documentation des méthodes d'un RDD est disponible ([RDD](https://spark.apache.org/docs/latest/api/python/reference/api/pyspark.RDD.html)).

1. Rappeler ce que sont les *transformations* et les *actions*
1. Donner, pour chaque prénom, son nombre d'occurences (`map` et `reduceByKey`)

In [6]:
#TODO
test = lignes.map(lambda l: (extract_field(l, 1),int(extract_field(l, 4))))

resultat = test.reduceByKey(lambda a, b: a + b)
for prenom, total in resultat.take(10):
    print(f"{prenom}: {total}")


ABDOULAYE: 7
ACHILLE: 9
ADAM: 113
ADEM: 4
ADRIEN: 151
AIDAN: 3
ALAN: 23
ALEX: 40
ALEXIS: 120
ALFRED: 29


1. Donner le nombre total de naissances avec un prénom féminin (`filter`, `map`, `reduce` ou `sum`)

In [7]:
#TODO
filtrer = lignes.filter(lambda l : l.startswith("2")== True)
traitement= filtrer.map(lambda l: int(extract_field(l, 4)))
resultat = traitement.reduce(lambda a,b: a+b)
print("total des filles", resultat)




total des filles 36039


1. Donner l'effectif maximal et minimal par prénom (`map`, `aggregateByKey`)

In [8]:
traitement= lignes.map(lambda l : (extract_field(l,1), int(extract_field(l,4))))
resultat = traitement.aggregateByKey((9999999,-999999999),
    lambda a,v: (min(a[0],v),max(a[1],v)),
    lambda acc1,acc2: (min(acc1[0],acc2[0]),max(acc1[1],acc2[1]))
)
resultat.take(10)

[('ABDOULAYE', (7, 7)),
 ('ACHILLE', (3, 6)),
 ('ADAM', (14, 50)),
 ('ADEM', (4, 4)),
 ('ADRIEN', (5, 52)),
 ('AIDAN', (3, 3)),
 ('ALAN', (3, 14)),
 ('ALEX', (3, 17)),
 ('ALEXIS', (3, 45)),
 ('ALFRED', (5, 12))]

1. Sur le modèle des prénoms, charger les données des départements
1. Donner, pour chaque nom de département, le prénom le plus fréquent depuis l'année 2000

In [9]:
lignes = sc.textFile("dpts.txt")
header = lignes.first()

lignes_sans_header = lignes.filter(lambda l: l != header)

departements = lignes_sans_header.map(lambda l: l.split(",")).map(lambda f: (
                         f[0],
                         int(f[1]),
                         f[2],
                         int(f[3]),
                         f[4],
                         f[5],
                         f[6]
                     ))

for d in departements.collect():
    print(d)

('01', 84, '01053', 5, 'AIN', 'Ain', 'Ain')
('02', 32, '02408', 5, 'AISNE', 'Aisne', 'Aisne')
('03', 84, '03190', 5, 'ALLIER', 'Allier', 'Allier')
('04', 93, '04070', 4, 'ALPES DE HAUTE PROVENCE', 'Alpes-de-Haute-Provence', 'Alpes-de-Haute-Provence')
('05', 93, '05061', 4, 'HAUTES ALPES', 'Hautes-Alpes', 'Hautes-Alpes')
('06', 93, '06088', 4, 'ALPES MARITIMES', 'Alpes-Maritimes', 'Alpes-Maritimes')
('07', 84, '07186', 5, 'ARDECHE', 'Ardèche', 'Ardèche')
('08', 44, '08105', 4, 'ARDENNES', 'Ardennes', 'Ardennes')
('09', 76, '09122', 5, 'ARIEGE', 'Ariège', 'Ariège')
('10', 44, '10387', 5, 'AUBE', 'Aube', 'Aube')
('11', 76, '11069', 5, 'AUDE', 'Aude', 'Aude')
('12', 76, '12202', 5, 'AVEYRON', 'Aveyron', 'Aveyron')
('13', 93, '13055', 4, 'BOUCHES DU RHONE', 'Bouches-du-Rhône', 'Bouches-du-Rhône')
('14', 28, '14118', 2, 'CALVADOS', 'Calvados', 'Calvados')
('15', 84, '15014', 2, 'CANTAL', 'Cantal', 'Cantal')
('16', 75, '16015', 3, 'CHARENTE', 'Charente', 'Charente')
('17', 75, '17300', 3, 'CH

In [23]:
prenom_plus_2000 = prenoms.filter(lambda l: l[2]>=2000)
departements_to_join = departements.map(lambda l: (l[0],l[4]))
prenom_plus_2000_to_join= prenom_plus_2000.map(lambda l: (str(l[3]),(l[1],l[4])))
join_prenom_dpt= prenom_plus_2000_to_join.join(departements_to_join)
join_prenom_dpt_named = join_prenom_dpt.map(
    lambda l: (l[1][1], l[1][0])
)

join_prenom_dpt_named.take(5)
resultat = join_prenom_dpt_named.aggregateByKey(("",0),
                    lambda acc,v: (v[0],v[1]) if (v[1]>acc[1]) else(acc[0],acc[1]),
                    lambda acc1,acc2: (acc1[0],acc1[1]) if (acc1[1]>acc2[1])else (acc2[0],acc2[1]))


for dep, (prenom, nb) in resultat.collect():
    print(f"{dep} : {prenom} ({nb})")





MARNE : ADAM (34)
COTE D OR : NOA (19)
LA REUNION : MATTÉO (33)
DROME : GABIN (23)
AVEYRON : ABEL (4)
VENDEE : ALEXIS (45)
ILLE ET VILAINE : LIAM (51)
MORBIHAN : CLARA (35)
MANCHE : EDGAR (7)
HAUTE SAVOIE : SARAH (35)
VOSGES : PAUL (30)
OISE : YANIS (40)
HAUTES PYRENEES : MILO (4)
LOZERE : JEANNE (3)
PAS DE CALAIS : NATHAN (85)
GIRONDE : _PRENOMS_RARES (437)
HERAULT : ALEX (17)
VAR : CLÉMENT (52)
SARTHE : CLÉMENT (31)
CHER : MATHÉO (11)
DOUBS : SALOMÉ (14)
GARD : ELENA (18)
COTES D ARMOR : ALEXIS (13)
LOIRET : JADE (35)
SAVOIE : AXEL (19)
SEINE ET MARNE : JULIE (77)
CHARENTE MARITIME : QUENTIN (32)
GUADELOUPE : LYAM (24)
AUBE : CÉLIA (13)
SEINE SAINT DENIS : RAYAN (122)
LOT ET GARONNE : ADAM (15)
LOIRE : ERWAN (14)
TARN : MATHÉO (19)
CREUSE : ENZO (10)
INDRE : THOMAS (27)
ESSONNE : VINCENT (46)
DEUX SEVRES : ARTHUR (23)
TERRITOIRE DE BELFORT : ADAM (14)
HAUTE GARONNE : ELIOTT (51)
PARIS : ALEXANDRE (344)
MEUSE : ANDY (4)
DORDOGNE : LÉO (22)
SOMME : CLARA (39)
ORNE : MANON (25)
CANTAL :